In [2]:
import pandas as pd 
import numpy as np
from citalopram_project.load import get_data_dir, load_neurons
from scipy.stats import mannwhitneyu


data_dir = get_data_dir()
derived_data_dir = get_data_dir() / "derived"
neurons = load_neurons().loc[lambda x: x.group != "discontinuation"]

In [3]:
def load_ensemble_dfs(derived_data_dir, name):
    stats = pd.read_parquet(derived_data_dir / f"{name}_stats.parquet.gzip")
    neurons = load_neurons().loc[lambda x: x.group != "discontinuation"]
    stats = stats.merge(neurons[["session_name", "group"]].drop_duplicates())
    ensembles = pd.read_parquet(derived_data_dir / f"{name}s.parquet.gzip")
    return stats, ensembles


def compare(df, value_col, group_col="group", groups=("sal", "continuation")):
    out = dict()
    x = df.loc[lambda x: x[group_col] == groups[0]][value_col]
    y = df.loc[lambda x: x[group_col] == groups[1]][value_col]
    out[f"n {groups[0]}"] = len(x)
    out[f"n {groups[1]}"] =len(y)
    out[f"m {groups[0]}"] = np.median(x)
    out[f"m {groups[1]}"] =np.median(y)
    out["U"], out["p"] = mannwhitneyu(x, y)
    out["p"] = round(out["p"], 3)
    return pd.Series(out)
    

In [4]:
spont_stats, spont_ensembles = load_ensemble_dfs(derived_data_dir, name="spont_ensemble") 

In [5]:
# number of ensembles per session

res = (
    spont_stats
    .groupby("session_name", as_index=False)["ensemble_sig"]
    .sum()
    .merge(neurons[["session_name", "group"]].drop_duplicates(), how="right")
    .pipe(compare, value_col="ensemble_sig")
)
print(res)

n sal              5.000
n continuation     7.000
m sal              2.000
m continuation     1.000
U                 16.000
p                  0.428
dtype: float64


In [6]:
# simmilarity, score of ensembles

res_sim = spont_stats.loc[lambda x: x.ensemble_sig].pipe(compare, value_col="simmilarity")
res_score = spont_stats.loc[lambda x: x.ensemble_sig].pipe(compare, value_col="score")

print(res_sim)
print(res_score)

n sal              8.000000
n continuation    11.000000
m sal              0.191198
m continuation     0.187652
U                 38.000000
p                  0.325000
dtype: float64
n sal              8.000000
n continuation    11.000000
m sal              0.140313
m continuation     0.125818
U                 40.000000
p                  0.386000
dtype: float64


In [18]:
def get_slow_fs_response(derived_data_dir):
    return pd.read_csv(derived_data_dir / "footshock_slow_timescale.csv").assign(
        slow_fs_response=lambda x: np.where(
            x.sig, np.where(x.statistic >= 0, "inhibited", "activated"), "no effect"
        )
    )[["neuron_id", "slow_fs_response"]]


def get_fast_fs_response(derived_data_dir):
    return pd.read_csv(derived_data_dir / "footshock_fast_timescale.csv").assign(
        fast_fs_response=lambda x: np.where(
            x.sig,
            np.where(x["Observed Difference"] >= 0, "inhibited", "activated"),
            "no effect",
        )
    )[["neuron_id", "fast_fs_response"]]


def get_eeg_val(derived_data_dir):
    return pd.read_csv(derived_data_dir / "eeg_corr.csv").assign(
        eeg_val=lambda x: np.where(x.eeg_corr < 0, "negative", "positive")
    )[["neuron_id", "eeg_val"]]


def get_eeg_mag(derived_data_dir):
    return pd.read_csv(derived_data_dir / "eeg_corr.csv")[["neuron_id", "abs_eeg_corr"]]

In [19]:
# get_props_by_neuron

neuron_props = neurons[["neuron_id", "session_name", "group", "cluster"]]

slow_fs = get_slow_fs_response(derived_data_dir)

fast_fs = get_fast_fs_response(derived_data_dir)

eeg_val = get_eeg_val(derived_data_dir)

eeg_mag = get_eeg_mag(derived_data_dir)

In [20]:
neuron_props = neurons[["neuron_id", "session_name", "group", "cluster"]]
neuron_props.merge(slow_fs).merge(fast_fs).merge(eeg_val).merge(eeg_mag)

,neuron_id,session_name,group,cluster,slow_fs_response,fast_fs_response,eeg_val,abs_eeg_corr
0,1069,hamilton_01,sal,slow_irregular,no effect,activated,negative,0.145050
1,1070,hamilton_01,sal,slow_irregular,activated,activated,negative,0.092257
2,1071,hamilton_01,sal,slow_irregular,activated,activated,negative,0.134405
3,1072,hamilton_01,sal,fast_firing,no effect,inhibited,negative,0.084804
4,1074,hamilton_01,sal,slow_irregular,no effect,activated,positive,0.026738
...,...,...,...,...,...,...,...,...
211,1605,hamilton_20,continuation,slow_irregular,activated,no effect,negative,0.202054
212,1606,hamilton_20,continuation,slow_irregular,activated,activated,negative,0.040198
213,1609,hamilton_20,continuation,slow_regular,inhibited,inhibited,negative,0.222604
214,1610,hamilton_20,continuation,slow_irregular,inhibited,no effect,negative,0.232238
